In [1]:
# import packages
import pandas as pd

In [2]:
df = pd.read_csv(r'data/log_fw_3.csv', sep=';', header=None)

#### data analysis

In [10]:
df.head(5)
df.describe()
df.describe(include='O')

,0,1,2,3,7,8
count,1000000,999998,999998,999998,999998,999998
unique,285454,31296,1,1,2,1
top,2023-02-28 12:37:41,109.234.162.235,17.17.17.17,TCP,PERMIT,eth0
freq,4374,542741,999998,999998,764709,999998


#### data cleaning

In [13]:
df.drop([8,9], axis=1, inplace=True)
cnames = ['datetime','ipsrc','ipdst','proto','portsrc','portdst','policyid','action','numproto']
df.columns = cnames
df['policyid'] = df['policyid'].values.astype('int')
df['policyid'] = df['policyid'].values.astype('str')

Now that we have a clean dataframe, we will create some new dataframe to build visualisation
#### Create a new dataframe named rules based on the proportion of each policyid

In [14]:
rules = pd.DataFrame(df['policyid'].value_counts(normalize=True))
rules.reset_index(inplace=True)
rules.columns = ['policyid','percentage']
threshold = 0.2
rules = rules[rules['percentage'] > threshold]
rules['percentage'] = rules['percentage']*100

## Rules ranking by their utility

In [39]:
''' plot bar with plotly express most used rules '''
import plotly.express as px
fig = px.bar(rules, x='policyid', y='percentage', title='Most used rules')

fig.update_layout(template='plotly_dark')

fig.show()

Here we have a bar chart, that show us the most used rules with their percentage and names.

#### Create a new dataframe named UDPrules based on the proportion of each policyid of UDP

In [17]:
# Transformation of the data for the analysis
UDPrules = pd.DataFrame(df['policyid'][df['proto'] == 'UDP'].value_counts(normalize=True))
UDPrules.reset_index(inplace=True)
UDPrules.columns = ['policyid','percentage']
UDPrules10 = UDPrules.head(10)
UDPrules10['percentage'] = UDPrules10['percentage']*100

## Top 10 rules UDP

In [18]:
''' plot bar with plotly express 10 most used rules with UDP protocol '''
import plotly.express as px
fig = px.bar(UDPrules10, x='policyid', y='percentage', title='Most 10 used rules with UDP protocol')

fig.update_layout(template='plotly_dark')

fig.show()

#### Create a new dataframe named UDPrules based on the proportion of each policyid of UDP

In [19]:
# Transformation of the data for the analysis
TCPrules = pd.DataFrame(df['policyid'][df['proto'] == 'TCP'].value_counts(normalize=True))
TCPrules.reset_index(inplace=True)
TCPrules.columns = ['policyid','percentage']
TCPrules5 = TCPrules.head(5)
TCPrules5['percentage'] = TCPrules5['percentage']*100

<ipython-input-19-b89a1d39161b>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## TOP 5 rules TCP

In [21]:
''' plot bar with plotly express 5 most used rules with TCP protocol '''
import plotly.express as px
fig = px.bar(TCPrules5, x='policyid', y='percentage', title='Most 5 used rules with TCP protocol')

fig.update_layout(template='plotly_dark')

fig.show()

Now, we want to know if we have more package at a time of the day. We will use a bar polar.

In [22]:
# extract time from datetime
time = df['datetime'].str.split(' ').str[1]
df['hour'] = time.str.split(':').str[0]

In [24]:
# count the number of occurences of each hour
h = pd.DataFrame(df['hour'].value_counts(normalize=False))
h.reset_index(inplace=True)
h.columns = ['hour','count']
h.sort_values(by=['hour'], inplace=True)

## Number of packages per hour

In [26]:
'''bar polar chart with plotly express for count of hour.'''
import plotly.express as px

fig = px.bar_polar(h, r='count', theta='hour', color='hour', title='proportion of package per hour', color_discrete_sequence=px.colors.sequential.Plasma_r)

fig.update_layout(polar = dict(
                                radialaxis = dict(showticklabels=False, ticks = ''),
                                angularaxis = dict(showticklabels=True, ticks = '')
                               ),
                template='plotly_dark',
                showlegend=False)


fig.show()

We can see that we have a lot of packages at 1 am. The other peak is at 12 am but he is less relevant.

In [27]:
# Here we transform and create dataframes for the analysis
df_deny = df[df['action'] == 'DENY']
df_permit = df[df['action'] == 'PERMIT']

top_ports_deny = df_deny['portdst'].value_counts().nlargest(10).index.tolist()
df_deny = df_deny[df_deny['portdst'].isin(top_ports_deny)]
top_ports_permit = df_permit['portdst'].value_counts().nlargest(10).index.tolist()
df_permit = df_permit[df_permit['portdst'].isin(top_ports_permit)]

df_deny_gb = df_deny.groupby(['policyid', 'portdst']).size().reset_index(name='counts')
df_permit_gb = df_permit.groupby(['policyid', 'portdst']).size().reset_index(name='counts')

df_action = pd.DataFrame(df.value_counts(['action','policyid']))
df_action.reset_index(inplace=True)
df_action.columns = ['action','policyid','counts']


## Top Rules by action
Here we want to know what rule define the permission or denied access.


In [37]:
action_sunburst = px.sunburst(df_action, path=['action', 'policyid'], values='counts', title='Policyid by action')

action_sunburst.update_layout(
template='plotly_dark')

action_sunburst.show()

So, as we can see, there are 2 rules that dominate the others for PERMIT action and 1 rule for the DENY action.

## Ports that rules Permit

In [35]:
permit_sunburst = px.sunburst(df_permit_gb, path=['policyid', 'portdst'], values='counts', title='Top ports by policyid from PERMIT action')

permit_sunburst.update_layout(
template='plotly_dark')

permit_sunburst.show()

## Top ports that rules Denied

We could see that 999 rule is massively present than the 7 rule, but the rule 7 bloc only the port 23.

In [38]:
deny_sunburst = px.sunburst(df_deny_gb, path=['policyid', 'portdst'], values='counts')

deny_sunburst.update_layout(
template='plotly_dark')

deny_sunburst.show()